In [162]:
import os
import json

In [163]:
endpoint_namespace = input("enter enpoint namespace: ")
endpoint_route = input("enter enpoint route: ")

endpoint_namespace + '/' + endpoint_route

'payments/methods'

In [164]:
route_singular = input("enter singular form of the route: ")
route_singular

'method'

In [165]:
directory = f'app/controllers/{endpoint_namespace}'

if not os.path.exists(directory):
  os.makedirs(directory)

In [166]:
filename = endpoint_route + '_controller.rb'

In [167]:
filepath = os.path.join(directory, filename)
filepath

'app/controllers/payments/methods_controller.rb'

In [168]:
route_actions = input('Actions to be added (CRUD): ').upper()
route_actions

'CRUD'

In [169]:
if 'C' in route_actions or 'U' in route_actions:
  object_params = input('input the payload format of creating or updating the object, as a dictionary: ')
  # object_params must be input in proper format for conversion to json dictionary
  # for ex.: {'transactions': ['method_id', 'category_id', 'type', 'amount', 'memo']}
  # the '' commas around each word are important, or else JSON won't be able to parse
  # Can also use "" instead of ''
  object_params

In [170]:
print('obj before', object_params)
object_params = object_params.replace("'", '"')
object_params = json.loads(object_params)
print('obj after', object_params)

obj before {'methods': ['icon_name', 'name']}
obj after {'methods': ['icon_name', 'name']}


In [171]:
if 'R' in route_actions:
  filter_params = input('input the comma-separated filter params: ') # Ex.: :method_id, :type, :category_id, :start_date, :end_date
  sort_params = input('input the comma-separated sort params: ') # Ex. : :amount, :date
  print(filter_params)
  print(sort_params)

In [172]:
module_name = '::' + endpoint_namespace.capitalize() + '::' + endpoint_route.capitalize()
module_name

'::Payments::Methods'

In [173]:
controller_name = module_name + 'Controller'
controller_name

'::Payments::MethodsController'

In [174]:
list(object_params.keys())

['methods']

In [175]:
if 'filter_param' in globals():
  print(1)

In [176]:
lines = [
"# frozen_string_literal: true",
"\n",
f"class {controller_name} < ApplicationController",
f"before_action :set_{route_singular}, only: [:show, :update, :destroy]\n"
]


index_function_lines=[
     "def index",
     f'''use_case = {module_name}::IndexCase.new(
      filter_parameters: {filter_params},
      sort_params: {sort_params},
      page_parameters: page_parameters)\n''',
      "result = use_case.perform\n",
      f'''render json: result.items,
                    meta: result.pagination,
                    meta_key: :pagination,
                    each_serializer: {module_name}Serializer''',
      "end\n"
]

show_function_lines = [
  "def show",
  f"use_case = {module_name}::ShowCase.new(account: @account, id: @{route_singular}.id)\n",
  "result = use_case.perform\n",
  f"render json: result, serializer: {module_name}Serializer",
  "end\n"
]

create_function_lines = [
  "def create",
  f"use_case = {module_name}::CreateCase.new({route_singular}_params.merge(account: @account))\n",
  "result = use_case.perform\n",
  "if result.errors.any?",
  "render json: { errors: result.errors }, status: :unprocessable_entity",
  "else",
  f"render json: result.record, serializer: {module_name}Serializer, status: :created",
  "end",
  "end\n"
]

update_function_lines = [
  "def update",
  f"use_case = {module_name}::UpdateCase.new({route_singular}_params.merge(id: @{route_singular}.id, account: @account))\n",
  "result = use_case.perform\n",
  "if result.errors.any?",
  "render json: { errors: result.errors }, status: :unprocessable_entity",
  "else",
  f"render json: result.record, serializer: {module_name}Serializer, status: :ok",
  "end",
  "end\n"
]

destroy_function_lines = [
  "def destroy",
  f"use_case = {module_name}::DestroyCase.new(id: @{route_singular}.id, account: @account)\n",
  "use_case.perform\n",
  "if use_case.errors.any?",
  "render json: { errors: use_case.errors }, status: :unprocessable_entity",
  "else",
  "head: :no_content",
  "end",
  "end\n"
]

if "R" in route_actions:
  for line in index_function_lines:
     lines.append(line)

if "C" in route_actions:
  for line in create_function_lines:
     lines.append(line)

if "R" in route_actions: # preserving order of index, create, show, so checking for 'R' twice
  for line in show_function_lines:
     lines.append(line)

if "U" in route_actions:
  for line in update_function_lines:
     lines.append(line)

if "D" in route_actions:
  for line in destroy_function_lines:
    lines.append(line)

lines.append("private\n")

object_param_items = ''
for idx, item in enumerate(list(object_params.values())[0]):
  object_param_items += f':{item}'

  if idx != len(list(object_params.values())[0]) - 1:
    object_param_items += ', '

object_param_lines = [
   f"def {route_singular}_params",
   f'''params.require(:{list(object_params.keys())[0]})
             .permit({object_param_items})''',
   "end\n"
]

if "C" in route_actions or "U" in route_actions:
    for line in object_param_lines:
      lines.append(line)

filter_param_lines = [
   "def filter_params",
   f"params.permit({filter_params})",
   "end\n"
]

if "filter_params" in globals() and len(filter_params) > 0:
  for line in filter_param_lines:
    lines.append(line)

sort_param_lines = [
   "def sort_parameters",
   f"params.permit(sort: [{sort_params}])",
   "end\n"
]

if "sort_params" in globals() and len(sort_params) > 0:
  for line in sort_param_lines:
    lines.append(line)

set_route_singular_lines = [
   f"def set_{route_singular}",
   f"@{route_singular} = @account.{endpoint_route}.find(params[:id])",
   "end"
]

for line in set_route_singular_lines:
     lines.append(line)

lines.append("end")


In [177]:
with open(filepath, 'w') as file:
  for line in lines:
    file.write(f"{line}\n")
  